## Shell Commands for the telescope

In [13]:
import astropy
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u
from datetime import datetime, timedelta
from astropy.coordinates import get_sun, get_moon
import sncosmo
from astropy.coordinates import Angle
from astropy.io import fits

import numpy as np 
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.io import fits
from astropy.modeling.models import Sersic2D
from astropy.modeling.models import Sersic1D
import os
import glob
import pandas as pd
np.set_printoptions(threshold=np.inf)
SMALL_SIZE = 10
plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
np.set_printoptions(threshold=np.inf)
import healpy as hp

from scipy.optimize import leastsq
import wget

import uncertainties as unc

In [69]:
# Reading the list in 

def read_target_list(filename):

    """
    Function to read a target list from a .csv file and add the SkyCoord objects as a new column. \n
    It further renames the columns "TARGET_RA" and "TARGET_DEC" to "RA" and "dec"
 
    Parameters
    ----------

    filename: str
        filename is the path specifying where to get the target list as a .csv file from

    Returns
    -------

    file: pd.DataFrame
        target list containing the .csv files information and the added SkyCoords as a pandas data frame object
    """

    # Read from a file
    print("Reading csv table...")
    file = pd.read_csv(filename)
    
    # Rename columns
    file = file.rename(columns = {"TARGET_RA": "RA", "TARGET_DEC": "dec"})

    # Calculate and add SkyCoords for AltAz calculation later
    print("Adding SkyCoords to it...")
    sky_coords = SkyCoord(file["RA"], file["dec"], unit = "deg")

    file["SKY_COORD"] = sky_coords
    
    #Return the Pandas dataframe (i.e. The unmodified initial target list)
    print("Finished, now returning pandas df")
    return file

In [68]:
superevent_name = 'S230627c'
target_list = read_target_list(f'Galaxy Selection/PGAL/PGAL_{superevent_name}_crossmatch_99.ecsv')
ra = target_list['RA']
dec = target_list['dec']
# print(dec)


Reading csv table...
Adding SkyCoords to it...
Finished, now returning pandas df


## Downloading the legacy files for the candidates

In [11]:
# Enter filter 
filters = 'g'
instrument = '3kk_ugriz'
size = {'3kk_nir': 1680, '3kk_ugriz': 2100, 'wwfi': 6000}
pixscale = {'3kk_nir': 0.25, '3kk_ugriz':0.2, 'wwfi':0.2}
# directory = f'../../../cfs/cdirs/desi/science/td/wendelstein/{superevent_name}_legacy_images/'
directory = f'../../../../cfs/cdirs/desi/science/td/wendelstein/{superevent_name}_legacy_images/'
if not os.path.exists(directory):
    os.makedirs(directory)
# Change the number of candidates here!
for i in range(len(ra[:10])):
    url_legacy = f"https://www.legacysurvey.org/viewer/fits-cutout?ra={round(ra[i],4)}&dec={round(dec[i],4)}&layer=ls-dr10&pixscale={pixscale[instrument]}&bands={filters}&size={size[instrument]}"
    file = wget.download(url_legacy, out = directory)

                                                                        

## Printing out commands for the telescope

In [57]:
def print_3kk_command(ra, dec, eventname, bfilter, rfilter, nirfilter, texp, numobs):
    text = ''
    # if(dec<0):
    #     print("not sure I am handling negative dec right")
    
    dec_a =  Angle(dec, u.degree)
    ra_a  =  Angle(ra, u.degree)
    text += "\n"+ "2mt " + str(int(texp)) 
    text += "\n"+ f"2mf b{bfilter} r{rfilter} n{nirfilter}"
    # Offset here for the 3KK to avoid objects in the chip gap
    for i in range(len(ra)):
        text += "\n" + "2mobj ligodesi_" + superevent_name + "_" + "{:02d}".format(int(ra_a[i].hms.h)) + "{:02d}".format(int(ra_a[i].hms.m)) + "{:02d}".format(int(ra_a[i].hms.s)) + "_" + "{:02d}".format(int(dec_a[i].dms.d)) + "{:02d}".format(int(dec_a[i].dms.m)) + "{:02d}".format(int(dec_a[i].dms.s)) + " 3441"
        ra_a[i] = ra_a[i] + Angle(35/np.cos(np.deg2rad(dec[i])) * u.deg/3600)
        text += "\n"+ "2mradec " + str(int(ra_a[i].hms.h)) + " " + str(int(ra_a[i].hms.m)) + " " + "{:2.2f}".format(ra_a[i].hms.s) + "   " + str(int(dec_a[i].dms.d)) + " " + str(int(dec_a[i].dms.m)) + " " + "{:2.2f}".format(dec_a[i].dms.s) 
        

        text += "\n"+ "2mdithersize 10"
        text += "\n"+ "2mh best"
        # print("2mh calcz")
        text += "\n"+ "2mautofocus"
        
        text += "\n"+ "2mo " + str(int(numobs))
        text += "\n"+ f"#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra={round(ra_a[i].degree,4)}&dec={round(dec_a[i].degree,4)}&layer=ls-dr10&pixscale=0.262"
        text += "\n"+ f"#wget https://www.legacysurvey.org/viewer/fits-cutout?ra={round(ra_a[i].degree,4)}&dec={round(dec_a[i].degree,4)}&layer=ls-dr10&pixscale=0.262&bands=griz"
    return(text)


In [58]:
text_3kk = print_3kk_command(ra, dec , 'S230627c', 'g', 'i', 'J', 90, 5)
print(text_3kk)
filename = f"{superevent_name}_redmine_commands.sh"

directory = f'Shell Commands/{superevent_name}_redmine_commands/'
if not os.path.exists(directory):
    os.makedirs(directory)
file_path = directory + '/' + filename    
with open(file_path, 'w') as file:
        file.writelines(text_3kk)


2mt 90
2mf bg ri nJ
2mobj ligodesi_S230627c_103833_465807 3441
2mradec 10 38 36.91   46 58 7.31
2mdithersize 10
2mh best
2mautofocus
2mo 5
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=159.6538&dec=46.9687&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=159.6538&dec=46.9687&layer=ls-dr10&pixscale=0.262&bands=griz
2mobj ligodesi_S230627c_104722_503304 3441
2mradec 10 47 26.64   50 33 4.36
2mdithersize 10
2mh best
2mautofocus
2mo 5
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=161.861&dec=50.5512&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=161.861&dec=50.5512&layer=ls-dr10&pixscale=0.262&bands=griz
2mobj ligodesi_S230627c_103244_391322 3441
2mradec 10 32 47.22   39 13 22.74
2mdithersize 10
2mh best
2mautofocus
2mo 5
#open https://www.legacysurvey.org/viewer/jpeg-cutout?ra=158.1967&dec=39.223&layer=ls-dr10&pixscale=0.262
#wget https://www.legacysurvey.org/viewer/fits-cutout?ra=158.1967&d

## Printing out commands for the coadds

In [36]:
# Function to print out terminal commands for coadds

def print_coadd_commands(ra, dec, eventname, filter):
    text = ''
    ra = target_list['RA'][:10]
    dec = target_list['dec'][:10]
    eventname = superevent_name
    CCD = {'i':1, 'r': 0}
    ra_a = Angle(ra, u.degree)
    dec_a =  Angle(dec, u.degree)
    num = ''
    for x in superevent_name:
        if x.isdigit():
            num += str(x)
    for i in range(len(ra)):
        text += "\n"+ "WORKDIR=/data/wst/u/ps1ipp/arri/3kk"
        text += "\n"+ "mkdir $WORKDIR/bias"
        text += "\n"+ "mkdir $WORKDIR/dark"
        text += "\n"+ "mkdir $WORKDIR/flat"
        text += "\n"+ "cd $WORKDIR"
        text += "\n"+ f"/data/wst/u/ps1ipp/arri/3kk/script_3kk_ligodesi.sh  ligodesi_{superevent_name}" + "_" + "{:02d}".format(int(ra_a[i].hms.h)) + "{:02d}".format(int(ra_a[i].hms.m)) + "{:02d}".format(int(ra_a[i].hms.s))+ "_" + "{:02d}".format(int(dec_a[i].dms.d)) + "{:02d}".format(int(dec_a[i].dms.m)) + "{:02d}".format(int(dec_a[i].dms.s)) + '  ' +"20"+ num + " " + f"{filter}"
        text += "\n"+ f"cd ligodesi_{superevent_name}" + "_" + "{:02d}".format(int(ra_a[i].hms.h)) + "{:02d}".format(int(ra_a[i].hms.m)) + "{:02d}".format(int(ra_a[i].hms.s)) + "_" + "{:02d}".format(int(dec_a[i].dms.d)) + "{:02d}".format(int(dec_a[i].dms.m)) + "{:02d}".format(int(dec_a[i].dms.s)) + "/" + "20" + num + "_" + f"{filter}"
        text += "\n"+ f"/data/wst/u/ps1ipp/arri/3kk/script_3kk_header_for_astrometry.sh  RA={ra_a[i].deg}  DEC={dec_a[i].deg}  CCD={CCD[filter]}  mepdnybo-n2023*.fits"
        text += "\n"+ f"sethead  FILTER={filter} mepdnybo-n*.fits"
        text += "\n"+ "for i in mepdnybo-*.fits; do  script.astrometry.sh $i 1;  done"
        text += "\n"+ "FIRST=`echo vme*.fits | awk '{print $1}'`"
        text += "\n" + "divvy  -e          -2  -3  -wcsok  -ref ${FIRST}    vme*.fits "
        text += "\n"+ "sumfits -n tvvmepdnybo-n2023*.fits" + "\n"
    return(text)

text = print_coadd_commands(target_list['RA'], target_list['dec'], 'S230615az', 'i')
filename = f"{superevent_name}_3kk_commands.sh"

directory = f'Shell Commands/{superevent_name}_commands/'
if not os.path.exists(directory):
    os.makedirs(directory)
file_path = directory + '/' + filename    
with open(file_path, 'w') as file:
        file.writelines(text)